In [ ]:
import pandas as pd
import numpy as np
import missingno as mno
import zipfile
import plotly.graph_objects as go
from collections import defaultdict
from plotly.subplots import make_subplots
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/time series/nytaxi_demand/train_distances_weather.csv')
df

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,...,visibility,solarradiation,solarenergy,uvindex,severerisk,conditions,icon,stations,datetime_y,description
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,...,8.5,30.3,0.1,0.0,NaN,"Rain, Overcast",rain,"72505394728,KLGA,KNYC,72503014732",2016-03-14,Cloudy skies throughout the day with a chance ...
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,...,16.0,0.0,0.0,0.0,NaN,Clear,clear-night,"72505394728,KLGA,KNYC,72503014732",2016-06-12,Partly cloudy throughout the day.
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,...,16.0,471.5,1.7,5.0,NaN,Clear,clear-day,"72505394728,KLGA,KNYC,72503014732",2016-01-19,Clear conditions throughout the day.
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,...,16.0,3.3,0.0,0.0,NaN,Partially cloudy,partly-cloudy-day,"72505394728,KLGA,KNYC,72503014732",2016-04-06,Becoming cloudy in the afternoon.
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,...,16.0,841.5,3.0,8.0,NaN,Clear,clear-day,"72505394728,KLGA,KNYC,72503014732",2016-03-26,Becoming cloudy in the afternoon.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1458639,id2376096,2,2016-04-08 13:31:04,2016-04-08 13:44:02,4,-73.982201,40.745522,-73.994911,40.740170,N,...,16.0,648.0,2.3,6.0,NaN,Overcast,cloudy,"72505394728,KLGA,KNYC,72503014732",2016-04-08,Partly cloudy throughout the day with morning ...
1458640,id1049543,1,2016-01-10 07:35:15,2016-01-10 07:46:10,1,-74.000946,40.747379,-73.970184,40.796547,N,...,3.5,11.1,0.0,0.0,NaN,"Rain, Overcast",rain,"72505394728,KLGA,KNYC,72503014732",2016-01-10,Partly cloudy throughout the day with a chance...
1458641,id2304944,2,2016-04-22 06:57:41,2016-04-22 07:10:25,1,-73.959129,40.768799,-74.004433,40.707371,N,...,16.0,26.1,0.1,0.0,NaN,Partially cloudy,partly-cloudy-night,"72505394728,KLGA,KNYC,72503014732",2016-04-22,Partly cloudy throughout the day.
1458642,id2714485,1,2016-01-05 15:56:26,2016-01-05 16:02:39,1,-73.982079,40.749062,-73.974632,40.757107,N,...,16.0,154.5,0.6,2.0,NaN,Clear,clear-day,"72505394728,KLGA,KNYC,72503014732",2016-01-05,Clear conditions throughout the day.


In [ ]:
pd.DataFrame({'Dtype': df.dtypes, 'Nunique': df.nunique(), 'Isnull': df.isnull().sum()}, index=df.columns)

,Dtype,Nunique,Isnull
temp,float64,386,0
feelslike,float64,498,0
dew,float64,430,0
humidity,float64,3035,0
precip,float64,234,0
precipprob,float64,2,0
snow,float64,7,0
snowdepth,float64,335,0
windspeed,float64,317,0
winddir,float64,355,0


In [ ]:
df[df['temp'].isnull()]

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,...,visibility,solarradiation,solarenergy,uvindex,severerisk,conditions,icon,stations,datetime_y,description
135282,id0253317,1,2016-03-13 02:03:49,2016-03-13 02:14:13,1,-73.992035,40.74913,-73.964355,40.757912,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-03-13,Becoming cloudy in the afternoon.


In [ ]:
nan_col = ['preciptype','windgust','solarradiation','solarenergy','uvindex','severerisk']
uselss_col = ['alt1', 'alt2', 'id', 'dropoff_datetime','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude', 'sin_hourofday', 'cos_hourofday',
       'sin_dayofweek', 'cos_dayofweek', 'sin_dayofyear', 'cos_dayofyear',
       'pickup_datetime_hour', 'pickup_datetime_day','name','datetime_x','stations', 'datetime_y']
df.drop(columns=nan_col+uselss_col, inplace=True)
df

,vendor_id,pickup_datetime,passenger_count,store_and_fwd_flag,trip_duration,temp,feelslike,dew,humidity,precip,...,snow,snowdepth,windspeed,winddir,sealevelpressure,cloudcover,visibility,conditions,icon,description
0,2,2016-03-14 17:24:55,1,N,455,4.7,-0.7,2.4,84.94,0.220,...,0.0,0.0,34.0,67.0,1016.7,100.0,8.5,"Rain, Overcast",rain,Cloudy skies throughout the day with a chance ...
1,1,2016-06-12 00:43:35,1,N,663,26.0,26.0,18.4,62.59,0.000,...,0.0,0.0,1.4,34.0,1006.5,0.0,16.0,Clear,clear-night,Partly cloudy throughout the day.
2,2,2016-01-19 11:35:24,1,N,2124,-4.8,-13.9,-16.0,41.17,0.000,...,0.0,0.0,40.7,300.0,1016.1,7.2,16.0,Clear,clear-day,Clear conditions throughout the day.
3,2,2016-04-06 19:32:31,1,N,429,6.8,2.8,-3.7,46.99,0.000,...,0.0,0.0,26.1,180.0,1018.5,23.8,16.0,Partially cloudy,partly-cloudy-day,Becoming cloudy in the afternoon.
4,2,2016-03-26 13:30:55,1,N,435,9.8,8.0,-1.4,45.60,0.000,...,0.0,0.0,12.5,62.0,1026.9,13.4,16.0,Clear,clear-day,Becoming cloudy in the afternoon.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1458639,2,2016-04-08 13:31:04,4,N,778,8.5,5.9,-3.8,41.55,0.000,...,0.0,0.0,15.9,248.0,1002.5,97.0,16.0,Overcast,cloudy,Partly cloudy throughout the day with morning ...
1458640,1,2016-01-10 07:35:15,1,N,655,8.0,4.8,6.4,89.94,5.368,...,0.0,0.0,20.6,77.0,1005.3,100.0,3.5,"Rain, Overcast",rain,Partly cloudy throughout the day with a chance...
1458641,2,2016-04-22 06:57:41,1,N,764,17.1,17.1,9.8,62.39,0.000,...,0.0,0.0,2.5,61.0,1013.4,23.8,16.0,Partially cloudy,partly-cloudy-night,Partly cloudy throughout the day.
1458642,1,2016-01-05 15:56:26,1,N,373,-2.0,-6.0,-19.5,24.80,0.000,...,0.0,0.0,11.0,311.0,1035.3,0.0,16.0,Clear,clear-day,Clear conditions throughout the day.


In [ ]:
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'].dt.strftime('%Y-%m-%d %H')) #dates and hours only
#df.loc[(df['pickup_datetime']>'2016-04-05 14:00:00') & (df['pickup_datetime'] <= '2016-04-05 15:00:00')]
dates = df['pickup_datetime'].sort_values().unique() #sort, unique
df[['trips', 'passengers']] = [np.nan,np.nan]
for i in range(0, len(dates)-1):
    sub = df.loc[(df['pickup_datetime']>dates[i]) & (df['pickup_datetime'] <= dates[i+1]), 'passenger_count']
    df.loc[(df['pickup_datetime']>dates[i]) & (df['pickup_datetime'] <= dates[i+1]), ['trips','passengers']] = [len(sub), sub.sum()]
df

,vendor_id,pickup_datetime,passenger_count,store_and_fwd_flag,trip_duration,temp,feelslike,dew,humidity,precip,...,windspeed,winddir,sealevelpressure,cloudcover,visibility,conditions,icon,description,trips,passengers
0,2,2016-03-14 17:00:00,1,N,455,4.7,-0.7,2.4,84.94,0.220,...,34.0,67.0,1016.7,100.0,8.5,"Rain, Overcast",rain,Cloudy skies throughout the day with a chance ...,423.0,696.0
1,1,2016-06-12 00:00:00,1,N,663,26.0,26.0,18.4,62.59,0.000,...,1.4,34.0,1006.5,0.0,16.0,Clear,clear-night,Partly cloudy throughout the day.,449.0,783.0
2,2,2016-01-19 11:00:00,1,N,2124,-4.8,-13.9,-16.0,41.17,0.000,...,40.7,300.0,1016.1,7.2,16.0,Clear,clear-day,Clear conditions throughout the day.,402.0,662.0
3,2,2016-04-06 19:00:00,1,N,429,6.8,2.8,-3.7,46.99,0.000,...,26.1,180.0,1018.5,23.8,16.0,Partially cloudy,partly-cloudy-day,Becoming cloudy in the afternoon.,558.0,895.0
4,2,2016-03-26 13:00:00,1,N,435,9.8,8.0,-1.4,45.60,0.000,...,12.5,62.0,1026.9,13.4,16.0,Clear,clear-day,Becoming cloudy in the afternoon.,449.0,758.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1458639,2,2016-04-08 13:00:00,4,N,778,8.5,5.9,-3.8,41.55,0.000,...,15.9,248.0,1002.5,97.0,16.0,Overcast,cloudy,Partly cloudy throughout the day with morning ...,409.0,665.0
1458640,1,2016-01-10 07:00:00,1,N,655,8.0,4.8,6.4,89.94,5.368,...,20.6,77.0,1005.3,100.0,3.5,"Rain, Overcast",rain,Partly cloudy throughout the day with a chance...,111.0,188.0
1458641,2,2016-04-22 06:00:00,1,N,764,17.1,17.1,9.8,62.39,0.000,...,2.5,61.0,1013.4,23.8,16.0,Partially cloudy,partly-cloudy-night,Partly cloudy throughout the day.,221.0,326.0
1458642,1,2016-01-05 15:00:00,1,N,373,-2.0,-6.0,-19.5,24.80,0.000,...,11.0,311.0,1035.3,0.0,16.0,Clear,clear-day,Clear conditions throughout the day.,420.0,712.0


In [ ]:
df.loc[(df['pickup_datetime']>'2016-01-01T00:00:00.000000000') & (df['pickup_datetime'] <= '2016-01-01T01:00:00.000000000')]

,vendor_id,pickup_datetime,passenger_count,store_and_fwd_flag,trip_duration,temp,feelslike,dew,humidity,precip,...,windspeed,winddir,sealevelpressure,cloudcover,visibility,conditions,icon,description,trips,passengers
4943,1,2016-01-01 01:00:00,1,N,690,5.8,3.1,-3.3,52.23,0.0,...,13.0,300.0,1017.9,100.0,16.0,Overcast,cloudy,Partly cloudy throughout the day.,566.0,1115.0
8031,1,2016-01-01 01:00:00,2,N,476,5.8,3.1,-3.3,52.23,0.0,...,13.0,300.0,1017.9,100.0,16.0,Overcast,cloudy,Partly cloudy throughout the day.,566.0,1115.0
12245,2,2016-01-01 01:00:00,2,N,763,5.8,3.1,-3.3,52.23,0.0,...,13.0,300.0,1017.9,100.0,16.0,Overcast,cloudy,Partly cloudy throughout the day.,566.0,1115.0
13470,1,2016-01-01 01:00:00,1,N,546,5.8,3.1,-3.3,52.23,0.0,...,13.0,300.0,1017.9,100.0,16.0,Overcast,cloudy,Partly cloudy throughout the day.,566.0,1115.0
14287,1,2016-01-01 01:00:00,1,N,270,5.8,3.1,-3.3,52.23,0.0,...,13.0,300.0,1017.9,100.0,16.0,Overcast,cloudy,Partly cloudy throughout the day.,566.0,1115.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1451981,1,2016-01-01 01:00:00,2,N,178,5.8,3.1,-3.3,52.23,0.0,...,13.0,300.0,1017.9,100.0,16.0,Overcast,cloudy,Partly cloudy throughout the day.,566.0,1115.0
1452545,2,2016-01-01 01:00:00,6,N,1128,5.8,3.1,-3.3,52.23,0.0,...,13.0,300.0,1017.9,100.0,16.0,Overcast,cloudy,Partly cloudy throughout the day.,566.0,1115.0
1452903,2,2016-01-01 01:00:00,6,N,400,5.8,3.1,-3.3,52.23,0.0,...,13.0,300.0,1017.9,100.0,16.0,Overcast,cloudy,Partly cloudy throughout the day.,566.0,1115.0
1453442,2,2016-01-01 01:00:00,5,N,1127,5.8,3.1,-3.3,52.23,0.0,...,13.0,300.0,1017.9,100.0,16.0,Overcast,cloudy,Partly cloudy throughout the day.,566.0,1115.0


In [ ]:
df = df.groupby([df['pickup_datetime']]).last()
df

,vendor_id,passenger_count,store_and_fwd_flag,trip_duration,temp,feelslike,dew,humidity,precip,precipprob,...,windspeed,winddir,sealevelpressure,cloudcover,visibility,conditions,icon,description,trips,passengers
pickup_datetime,,,,,,,,,,,,,,,,,,,,,
2016-01-01 00:00:00,2,3,N,123,5.8,3.5,-2.9,53.85,0.0,0.0,...,10.4,320.0,1018.5,100.0,16.0,Overcast,cloudy,Partly cloudy throughout the day.,NaN,NaN
2016-01-01 01:00:00,2,2,N,928,5.8,3.1,-3.3,52.23,0.0,0.0,...,13.0,300.0,1017.9,100.0,16.0,Overcast,cloudy,Partly cloudy throughout the day.,566.0,1115.0
2016-01-01 02:00:00,2,4,N,1313,5.3,2.9,-3.4,53.48,0.0,0.0,...,10.9,320.0,1017.8,100.0,16.0,Overcast,cloudy,Partly cloudy throughout the day.,503.0,928.0
2016-01-01 03:00:00,2,1,N,1466,5.2,2.7,-3.2,54.94,0.0,0.0,...,11.0,296.0,1017.7,100.0,16.0,Overcast,cloudy,Partly cloudy throughout the day.,479.0,850.0
2016-01-01 04:00:00,2,3,N,375,5.2,1.8,-3.2,54.94,0.0,0.0,...,16.3,285.0,1017.7,100.0,16.0,Overcast,cloudy,Partly cloudy throughout the day.,323.0,615.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-06-30 19:00:00,1,2,N,1177,26.1,26.1,14.8,49.94,0.0,0.0,...,10.4,190.0,1015.8,50.0,16.0,Partially cloudy,partly-cloudy-day,Clear conditions throughout the day.,484.0,773.0
2016-06-30 20:00:00,1,1,N,439,26.1,26.1,15.8,53.13,0.0,0.0,...,11.8,200.0,1015.8,33.2,14.5,Partially cloudy,partly-cloudy-day,Clear conditions throughout the day.,440.0,733.0
2016-06-30 21:00:00,2,5,N,1702,25.3,25.3,16.3,57.46,0.0,0.0,...,7.9,190.0,1015.9,13.4,16.0,Clear,clear-night,Clear conditions throughout the day.,484.0,845.0


In [ ]:
df.dropna(inplace=True)
df.drop(columns=['vendor_id','passenger_count','store_and_fwd_flag','trip_duration'], inplace=True)
df

,temp,feelslike,dew,humidity,precip,precipprob,snow,snowdepth,windspeed,winddir,sealevelpressure,cloudcover,visibility,conditions,icon,description,trips,passengers
pickup_datetime,,,,,,,,,,,,,,,,,,
2016-01-01 01:00:00,5.8,3.1,-3.3,52.23,0.0,0.0,0.0,0.0,13.0,300.0,1017.9,100.0,16.0,Overcast,cloudy,Partly cloudy throughout the day.,566.0,1115.0
2016-01-01 02:00:00,5.3,2.9,-3.4,53.48,0.0,0.0,0.0,0.0,10.9,320.0,1017.8,100.0,16.0,Overcast,cloudy,Partly cloudy throughout the day.,503.0,928.0
2016-01-01 03:00:00,5.2,2.7,-3.2,54.94,0.0,0.0,0.0,0.0,11.0,296.0,1017.7,100.0,16.0,Overcast,cloudy,Partly cloudy throughout the day.,479.0,850.0
2016-01-01 04:00:00,5.2,1.8,-3.2,54.94,0.0,0.0,0.0,0.0,16.3,285.0,1017.7,100.0,16.0,Overcast,cloudy,Partly cloudy throughout the day.,323.0,615.0
2016-01-01 05:00:00,4.5,0.8,-3.0,58.21,0.0,0.0,0.0,0.0,17.6,270.0,1017.0,100.0,16.0,Overcast,cloudy,Partly cloudy throughout the day.,142.0,256.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-06-30 19:00:00,26.1,26.1,14.8,49.94,0.0,0.0,0.0,0.0,10.4,190.0,1015.8,50.0,16.0,Partially cloudy,partly-cloudy-day,Clear conditions throughout the day.,484.0,773.0
2016-06-30 20:00:00,26.1,26.1,15.8,53.13,0.0,0.0,0.0,0.0,11.8,200.0,1015.8,33.2,14.5,Partially cloudy,partly-cloudy-day,Clear conditions throughout the day.,440.0,733.0
2016-06-30 21:00:00,25.3,25.3,16.3,57.46,0.0,0.0,0.0,0.0,7.9,190.0,1015.9,13.4,16.0,Clear,clear-night,Clear conditions throughout the day.,484.0,845.0


In [ ]:
def cyclic_column_processing(df, dt_column):
    df[dt_column] = df[dt_column].astype('datetime64[ns]')
    df['hourofday'] = df[dt_column].dt.hour
    df['sin_hourofday'] = np.sin(2*np.pi*df['hourofday']/np.max(df['hourofday']))
    df['cos_hourofday'] = np.cos(2*np.pi*df['hourofday']/np.max(df['hourofday']))
    df.drop(columns=['hourofday'], inplace=True)

    df['dayofweek'] = df[dt_column].dt.dayofweek
    df['sin_dayofweek'] = np.sin(2*np.pi*df['dayofweek']/np.max(df['dayofweek']))
    df['cos_dayofweek'] = np.cos(2*np.pi*df['dayofweek']/np.max(df['dayofweek']))
    df.drop(columns=['dayofweek'], inplace=True)

    df['dayofyear'] = df[dt_column].dt.dayofyear
    df['sin_dayofyear'] = np.sin(2*np.pi*df['dayofyear']/np.max(df['dayofyear']))
    df['cos_dayofyear'] = np.cos(2*np.pi*df['dayofyear']/np.max(df['dayofyear']))
    df.drop(columns=['dayofyear'], inplace=True)
    return df

df['pickup_datetime'] = df.index
df = cyclic_column_processing(df, 'pickup_datetime')

In [ ]:
df

,temp,feelslike,dew,humidity,precip,precipprob,snow,snowdepth,windspeed,winddir,...,description,trips,passengers,pickup_datetime,sin_hourofday,cos_hourofday,sin_dayofweek,cos_dayofweek,sin_dayofyear,cos_dayofyear
pickup_datetime,,,,,,,,,,,,,,,,,,,,,
2016-01-01 01:00:00,5.8,3.1,-3.3,52.23,0.0,0.0,0.0,0.0,13.0,300.0,...,Partly cloudy throughout the day.,566.0,1115.0,2016-01-01 01:00:00,2.697968e-01,0.962917,-8.660254e-01,-0.5,3.451614e-02,0.999404
2016-01-01 02:00:00,5.3,2.9,-3.4,53.48,0.0,0.0,0.0,0.0,10.9,320.0,...,Partly cloudy throughout the day.,503.0,928.0,2016-01-01 02:00:00,5.195840e-01,0.854419,-8.660254e-01,-0.5,3.451614e-02,0.999404
2016-01-01 03:00:00,5.2,2.7,-3.2,54.94,0.0,0.0,0.0,0.0,11.0,296.0,...,Partly cloudy throughout the day.,479.0,850.0,2016-01-01 03:00:00,7.308360e-01,0.682553,-8.660254e-01,-0.5,3.451614e-02,0.999404
2016-01-01 04:00:00,5.2,1.8,-3.2,54.94,0.0,0.0,0.0,0.0,16.3,285.0,...,Partly cloudy throughout the day.,323.0,615.0,2016-01-01 04:00:00,8.878852e-01,0.460065,-8.660254e-01,-0.5,3.451614e-02,0.999404
2016-01-01 05:00:00,4.5,0.8,-3.0,58.21,0.0,0.0,0.0,0.0,17.6,270.0,...,Partly cloudy throughout the day.,142.0,256.0,2016-01-01 05:00:00,9.790841e-01,0.203456,-8.660254e-01,-0.5,3.451614e-02,0.999404
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-06-30 19:00:00,26.1,26.1,14.8,49.94,0.0,0.0,0.0,0.0,10.4,190.0,...,Clear conditions throughout the day.,484.0,773.0,2016-06-30 19:00:00,-8.878852e-01,0.460065,1.224647e-16,-1.0,-2.449294e-16,1.000000
2016-06-30 20:00:00,26.1,26.1,15.8,53.13,0.0,0.0,0.0,0.0,11.8,200.0,...,Clear conditions throughout the day.,440.0,733.0,2016-06-30 20:00:00,-7.308360e-01,0.682553,1.224647e-16,-1.0,-2.449294e-16,1.000000
2016-06-30 21:00:00,25.3,25.3,16.3,57.46,0.0,0.0,0.0,0.0,7.9,190.0,...,Clear conditions throughout the day.,484.0,845.0,2016-06-30 21:00:00,-5.195840e-01,0.854419,1.224647e-16,-1.0,-2.449294e-16,1.000000


In [ ]:
out_column = 'passengers'
to_remove = [] + [c for c in df.columns if df[c].nunique()==1]
features = [c for c in df.columns if (c not in [out_column]) and (c not in to_remove)]
categorical = [c for c in features if (df[c].dtype=='object') and (df[c].nunique() > 2)]
binary = [c for c in features if df[c].nunique() == 2]
numerical = [c for c in features if df[c].dtype=='float']

fig = make_subplots(rows=1, cols=2, horizontal_spacing = 0.03, subplot_titles=('Label Distribution', "Features Correlation"))
fig.append_trace(go.Histogram(x=df[out_column]), row=1, col=1)
fig.append_trace(go.Heatmap(z=df[numerical+[out_column]].corr(),x=numerical+[out_column],y=numerical+[out_column]), row=1, col=2)
fig.show()

In [ ]:
df.drop(columns=['feelslike'], inplace=True)
df.to_csv('/content/drive/My Drive/Colab Notebooks/time series/nytaxi_demand/nytaxi_demand.csv', index=False)